In [ ]:
con.close()

In [ ]:
import duckdb
import pandas as pd
import logging

def fetch_training_data(con, path, n=1000):
    """Fetches and structures data for training a model."""

    #-- Select a sample of matches for training
    complete_query ="""
    WITH match_data AS (
        SELECT 
            m.match_id,
            m.start_time,
            m.winning_team
        FROM 
            matches m
        ORDER BY 
            random()  -- Randomly select matches
        LIMIT 1000
    ),

    teams_of_players AS (
        SELECT
            pm.match_id,
            pm.account_id,
            pm.hero_id,
            pm.team,
            pm.kills,
            pm.deaths,
            pm.assists,
            CASE WHEN pm.deaths = 0 THEN pm.kills ELSE CAST(pm.kills AS FLOAT) / pm.deaths END AS kd_ratio
        FROM 
            player_matches pm
        JOIN 
            match_data md ON pm.match_id = md.match_id
    ),

    #-- Calculate team-level statistics

    team_stats AS (
        SELECT
            match_id,
            team,
            AVG(kd_ratio) AS team_avg_kd,
            MAX(kd_ratio) AS team_max_kd,
            MIN(kd_ratio) AS team_min_kd,
            COUNT(*) AS team_size
        FROM 
            teams_of_players
        GROUP BY 
            match_id, team
    ),

    #-- Get player trends and statistics
    player_performance AS (
        SELECT
            ta.match_id,
            ta.team,
            -- Basic performance stats
            AVG(pt.p_average_kills) AS avg_team_kills,
            AVG(pt.p_average_deaths) AS avg_team_deaths,
            AVG(pt.p_avg_kd) AS avg_team_kd,
            AVG(pt.p_win_rate) AS avg_team_win_rate,
            AVG(pt.p_total_matches) AS avg_player_matches,
            
            -- Streak information
            AVG(pt.p_win_streak_avg) AS avg_win_streak,
            AVG(pt.p_loss_streak_avg) AS avg_loss_streak,
            SUM(pt.win_streaks_2plus) AS team_win_streaks_2plus,
            SUM(pt.win_streaks_3plus) AS team_win_streaks_3plus,
            SUM(pt.win_streaks_4plus) AS team_win_streaks_4plus,
            SUM(pt.win_streaks_5plus) AS team_win_streaks_5plus,
            SUM(pt.loss_streaks_2plus) AS team_loss_streaks_2plus,
            SUM(pt.loss_streaks_3plus) AS team_loss_streaks_3plus,
            SUM(pt.loss_streaks_4plus) AS team_loss_streaks_4plus,
            SUM(pt.loss_streaks_5plus) AS team_loss_streaks_5plus,
            
            -- Hero comparison metrics
            AVG(pt.p_v_h_kd_pct) AS avg_hero_kd_percentage,
            
            -- Team strength indicators
            MAX(pt.p_win_rate) AS max_player_win_rate,
            MIN(pt.p_win_rate) AS min_player_win_rate,
            MAX(pt.p_avg_kd) AS max_player_kd,
            MIN(pt.p_avg_kd) AS min_player_kd,
            
            -- Experience metrics
            SUM(pt.p_total_matches) AS total_team_experience,
            MAX(pt.p_total_matches) AS most_experienced_player,
            MIN(pt.p_total_matches) AS least_experienced_player,
            
            -- Consistency metrics
            STDDEV(pt.p_win_rate) AS win_rate_consistency,
            STDDEV(pt.p_avg_kd) AS kd_consistency
        FROM 
            teams_of_players ta
        JOIN 
            player_trends pt ON ta.account_id = pt.account_id
        GROUP BY 
            ta.match_id, ta.team
    )

    #-- Calculate hero trend statistics by team
    WITH hero_trend_by_team AS (
        SELECT
            ta.match_id,
            ta.team,
            AVG(ht.win_rate) AS avg_hero_win_rate,
            AVG(ht.average_kd) AS avg_hero_kd,
            AVG(ht.pick_rate) AS avg_hero_pick_rate,
            SUM(ht.pick_rate) AS total_hero_popularity,
            MAX(ht.win_rate) AS max_hero_win_rate,
            MIN(ht.win_rate) AS min_hero_win_rate,
            MAX(ht.average_kd) AS max_hero_kd,
            MIN(ht.average_kd) AS min_hero_kd,
            STDDEV(ht.win_rate) AS hero_win_rate_variety,
            STDDEV(ht.average_kd) AS hero_kd_variety
        FROM 
            teams_of_players ta
        JOIN 
            hero_trends ht ON ta.hero_id = ht.hero_id
        WHERE 
            ht.trend_window_days = 30  -- Using 30-day trends
        GROUP BY 
            ta.match_id, ta.team
    ),

    #-- Get recent player performance (rolling stats)
    recent_performance AS (
        SELECT
            ta.match_id,
            ta.team,
            AVG(prs.p_win_pct_2) AS avg_recent_win_pct_2,
            AVG(prs.p_win_pct_3) AS avg_recent_win_pct_3,
            AVG(prs.p_win_pct_4) AS avg_recent_win_pct_4,
            AVG(prs.p_win_pct_5) AS avg_recent_win_pct_5,
            AVG(prs.p_loss_pct_2) AS avg_recent_loss_pct_2,
            AVG(prs.p_loss_pct_3) AS avg_recent_loss_pct_3,
            AVG(prs.p_loss_pct_4) AS avg_recent_loss_pct_4,
            AVG(prs.p_loss_pct_5) AS avg_recent_loss_pct_5,
            MAX(prs.p_win_pct_5) AS max_recent_win_pct,
            MIN(prs.p_win_pct_5) AS min_recent_win_pct
        FROM 
            teams_of_players ta
        JOIN 
            player_rolling_stats prs ON ta.account_id = prs.account_id AND ta.match_id = prs.match_id
        GROUP BY 
            ta.match_id, ta.team
    ),

    #-- create team features
    WITH team_features AS (
        SELECT
            ts.match_id,
            ts.team,
            -- Match-specific performance
            ts.team_avg_kd,
            ts.team_max_kd,
            ts.team_min_kd,
            
            -- Historical player performance
            pp.avg_team_kills,
            pp.avg_team_deaths,
            pp.avg_team_kd,
            pp.avg_team_win_rate,
            pp.max_player_win_rate,
            pp.min_player_win_rate,
            pp.max_player_kd,
            pp.min_player_kd,
            pp.avg_player_matches,
            pp.total_team_experience,
            pp.most_experienced_player,
            pp.least_experienced_player,
            pp.win_rate_consistency,
            pp.kd_consistency,
            
            -- Streak information
            pp.avg_win_streak,
            pp.avg_loss_streak,
            pp.team_win_streaks_2plus,
            pp.team_win_streaks_3plus,
            pp.team_win_streaks_4plus,
            pp.team_win_streaks_5plus,
            pp.team_loss_streaks_2plus,
            pp.team_loss_streaks_3plus,
            pp.team_loss_streaks_4plus,
            pp.team_loss_streaks_5plus,
            
            -- Hero comparison
            pp.avg_hero_kd_percentage,
            
            -- Hero performance
            ht.avg_hero_win_rate,
            ht.avg_hero_kd,
            ht.avg_hero_pick_rate,
            ht.total_hero_popularity,
            ht.max_hero_win_rate,
            ht.min_hero_win_rate,
            ht.max_hero_kd,
            ht.min_hero_kd,
            ht.hero_win_rate_variety,
            ht.hero_kd_variety,
            
            -- Recent performance
            COALESCE(rp.avg_recent_win_pct_2, 50) AS avg_recent_win_pct_2,
            COALESCE(rp.avg_recent_win_pct_3, 50) AS avg_recent_win_pct_3,
            COALESCE(rp.avg_recent_win_pct_4, 50) AS avg_recent_win_pct_4,
            COALESCE(rp.avg_recent_win_pct_5, 50) AS avg_recent_win_pct_5,
            COALESCE(rp.avg_recent_loss_pct_2, 50) AS avg_recent_loss_pct_2,
            COALESCE(rp.avg_recent_loss_pct_3, 50) AS avg_recent_loss_pct_3,
            COALESCE(rp.avg_recent_loss_pct_4, 50) AS avg_recent_loss_pct_4,
            COALESCE(rp.avg_recent_loss_pct_5, 50) AS avg_recent_loss_pct_5,
            COALESCE(rp.max_recent_win_pct, 50) AS max_recent_win_pct,
            COALESCE(rp.min_recent_win_pct, 50) AS min_recent_win_pct
        FROM 
            team_stats ts
        LEFT JOIN 
            player_performance pp ON ts.match_id = pp.match_id AND ts.team = pp.team
        LEFT JOIN 
            hero_trend_by_team ht ON ts.match_id = ht.match_id AND ts.team = ht.team
        LEFT JOIN 
            recent_performance rp ON ts.match_id = rp.match_id AND ts.team = rp.team
    ),

     -- Final dataset for model training
    SELECT
        md.match_id,
        md.start_time,
        md.winning_team,
        
        -- Team 0 features
        t0.team_avg_kd AS t0_avg_kd,
        t0.team_max_kd AS t0_max_kd,
        t0.team_min_kd AS t0_min_kd,
        t0.avg_team_kills AS t0_avg_kills,
        t0.avg_team_deaths AS t0_avg_deaths,
        t0.avg_team_kd AS t0_avg_historical_kd,
        t0.avg_team_win_rate AS t0_win_rate,
        t0.max_player_win_rate AS t0_max_win_rate,
        t0.min_player_win_rate AS t0_min_win_rate,
        t0.max_player_kd AS t0_max_historical_kd,
        t0.min_player_kd AS t0_min_historical_kd,
        t0.avg_win_streak AS t0_win_streak,
        t0.avg_loss_streak AS t0_loss_streak,
        t0.team_win_streaks_2plus AS t0_win_streaks_2,
        t0.avg_player_matches AS t0_player_matches,
        t0.total_team_experience AS t0_total_experience,
        t0.avg_hero_win_rate AS t0_hero_win_rate,
        t0.avg_recent_win_pct AS t0_recent_wins,
        
        -- Team 1 features
        t1.team_avg_kd AS t1_avg_kd,
        t1.team_max_kd AS t1_max_kd,
        t1.team_min_kd AS t1_min_kd,
        t1.avg_team_kills AS t1_avg_kills,
        t1.avg_team_deaths AS t1_avg_deaths,
        t1.avg_team_kd AS t1_avg_historical_kd,
        t1.avg_team_win_rate AS t1_win_rate,
        t1.max_player_win_rate AS t1_max_win_rate,
        t1.min_player_win_rate AS t1_min_win_rate,
        t1.max_player_kd AS t1_max_historical_kd,
        t1.min_player_kd AS t1_min_historical_kd,
        t1.avg_win_streak AS t1_win_streak,
        t1.avg_loss_streak AS t1_loss_streak,
        t1.team_win_streaks_2plus AS t1_win_streaks_2,
        t1.avg_player_matches AS t1_player_matches,
        t1.total_team_experience AS t1_total_experience,
        t1.avg_hero_win_rate AS t1_hero_win_rate,
        t1.avg_recent_win_pct AS t1_recent_wins,
        
        -- Differential features (team 0 relative to team 1)
        (t0.team_avg_kd - t1.team_avg_kd) AS kd_diff,
        (t0.avg_team_kd - t1.avg_team_kd) AS historical_kd_diff,
        (t0.avg_team_win_rate - t1.avg_team_win_rate) AS win_rate_diff,
        (t0.avg_win_streak - t1.avg_win_streak) AS win_streak_diff,
        (t0.avg_recent_win_pct - t1.avg_recent_win_pct) AS recent_win_diff,
        (t0.avg_hero_win_rate - t1.avg_hero_win_rate) AS hero_win_rate_diff,
        
        -- Target variable: 1 if team 0 won, 0 if team 1 won
        CASE WHEN md.winning_team = 'Team0' THEN 1 ELSE 0 END AS team0_won
    FROM 
        match_data md
    JOIN 
        team_features t0 ON md.match_id = t0.match_id AND t0.team = '0'
    JOIN 
        team_features t1 ON md.match_id = t1.match_id AND t1.team = '1'
    """

        # Execute the complete query
    try:
        result_df = pd.read_sql_query(complete_query, con)
        print(f"Successfully fetched {len(result_df)} matches for training")
        return result_df
    except Exception as e:
        print(f"Error executing query: {e}")
        # You might want to print the complete query here for debugging
        print(complete_query)
        return None

if __name__ == "__main__":
    con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")
    path = "C:/Code/Local Code/deadlock_match_prediction/data/test_data/"
    df = fetch_training_data(con, path)
    print(df.head())

In [1]:
import duckdb
import pandas as pd
import logging
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")

In [2]:
match_data = con.execute(f"""
    WITH match_data AS (
        SELECT 
            m.match_id,
            m.start_time,
            m.winning_team
        FROM 
            matches m
        ORDER BY 
            random()  -- Randomly select matches
        LIMIT 1000
    )
    SELECT * FROM match_data""").fetchdf()
print(match_data)

     match_id          start_time winning_team
0    34681243 2025-04-08 02:41:19        Team1
1    34754413 2025-04-10 21:00:55        Team0
2    35480510 2025-05-06 23:05:08        Team1
3    34515798 2025-04-02 16:27:11        Team0
4    33982008 2025-03-16 18:35:44        Team1
..        ...                 ...          ...
995  34011901 2025-03-17 17:19:24        Team1
996  34101828 2025-03-20 16:36:15        Team0
997  34221162 2025-03-24 01:03:46        Team1
998  34934160 2025-04-17 07:26:29        Team1
999  34104749 2025-03-20 18:18:08        Team0

[1000 rows x 3 columns]


In [3]:
team_of_players = con.execute(f"""
    WITH teams_of_players AS (
        SELECT
            pm.match_id,
            pm.account_id,
            pm.hero_id,
            pm.team,
            pm.kills,
            pm.deaths,
            pm.assists,
            CASE WHEN pm.deaths = 0 THEN pm.kills ELSE CAST(pm.kills AS FLOAT) / pm.deaths END AS kd_ratio
        FROM 
            player_matches pm
        JOIN 
            match_data md ON pm.match_id = md.match_id
    )
SELECT * FROM teams_of_players""").fetchdf()
print(team_of_players)


       match_id  account_id  hero_id   team  kills  deaths  assists  kd_ratio
0      34199713  1387549649       16  Team0      9      11       15  0.818182
1      34199713  1026126614       52  Team1     19       8       14  2.375000
2      34199713   108815945       13  Team0      6      10        9  0.600000
3      34199713  1122976129       14  Team1      7       6       16  1.166667
4      34199713  1477660209       15  Team1     14       9        9  1.555556
...         ...         ...      ...    ...    ...     ...      ...       ...
11995  34235632  1801125433       19  Team1     11       9        3  1.222222
11996  34235632  1139810324       31  Team0      9      12       13  0.750000
11997  34235632  1085036977        3  Team1      7       6       10  1.166667
11998  34235632  1101281569        4  Team1      6       7        7  0.857143
11999  34235632   204278132       11  Team1      2      14        5  0.142857

[12000 rows x 8 columns]


In [4]:

#-- Calculate team-level statistics
team_stats= con.execute(f"""
    WITH team_stats AS (
        SELECT
            match_id,
            team,
            AVG(kd_ratio) AS team_avg_kd,
            MAX(kd_ratio) AS team_max_kd,
            MIN(kd_ratio) AS team_min_kd,
            COUNT(*) AS team_size
        FROM 
            team_of_players
        GROUP BY 
            match_id, team
    )
    Select * from team_stats""").fetchdf()
print(team_stats)



      match_id   team  team_avg_kd  team_max_kd  team_min_kd  team_size
0     34216146  Team0     1.153902     2.000000     0.333333          6
1     34164057  Team0     1.128770     2.333333     0.214286          6
2     34164841  Team0     4.332011    13.000000     0.444444          6
3     34135998  Team1     1.306905     3.000000     0.409091          6
4     35537255  Team0     0.808135     1.428571     0.250000          6
...        ...    ...          ...          ...          ...        ...
1995  34289232  Team1     4.329762    18.000000     0.571429          6
1996  34264928  Team1     1.377778     2.333333     0.500000          6
1997  34266559  Team0     0.882053     2.125000     0.400000          6
1998  34275217  Team0     1.469048     3.500000     0.000000          6
1999  34277177  Team0     3.305556     6.500000     0.500000          6

[2000 rows x 6 columns]


In [5]:
player_performance = con.execute(f"""
    WITH player_performance AS (
        SELECT
            ta.match_id,
            ta.team,
            -- Basic performance stats
            AVG(pt.p_average_kills) AS avg_team_kills,
            AVG(pt.p_average_deaths) AS avg_team_deaths,
            AVG(pt.p_avg_kd) AS avg_team_kd,
            AVG(pt.p_win_rate) AS avg_team_win_rate,
            AVG(pt.p_total_matches) AS avg_player_matches,
            
            -- Streak information
            AVG(pt.p_win_streak_avg) AS avg_win_streak,
            AVG(pt.p_loss_streak_avg) AS avg_loss_streak,
            SUM(pt.win_streaks_2plus) AS team_win_streaks_2plus,
            SUM(pt.win_streaks_3plus) AS team_win_streaks_3plus,
            SUM(pt.win_streaks_4plus) AS team_win_streaks_4plus,
            SUM(pt.win_streaks_5plus) AS team_win_streaks_5plus,
            SUM(pt.loss_streaks_2plus) AS team_loss_streaks_2plus,
            SUM(pt.loss_streaks_3plus) AS team_loss_streaks_3plus,
            SUM(pt.loss_streaks_4plus) AS team_loss_streaks_4plus,
            SUM(pt.loss_streaks_5plus) AS team_loss_streaks_5plus,
            
            
            -- Team strength indicators
            MAX(pt.p_win_rate) AS max_player_win_rate,
            MIN(pt.p_win_rate) AS min_player_win_rate,
            MAX(pt.p_avg_kd) AS max_player_kd,
            MIN(pt.p_avg_kd) AS min_player_kd,
            
            -- Experience metrics
            SUM(pt.p_total_matches) AS total_team_experience,
            MAX(pt.p_total_matches) AS most_experienced_player,
            MIN(pt.p_total_matches) AS least_experienced_player,
            
            -- Consistency metrics
            STDDEV(pt.p_win_rate) AS win_rate_consistency,
            STDDEV(pt.p_avg_kd) AS kd_consistency
        FROM 
            team_of_players ta
        JOIN 
            player_trends pt ON ta.account_id = pt.account_id
        GROUP BY 
            ta.match_id, ta.team
    )
    SELECT * FROM player_performance
    """).fetchdf()
print(player_performance)

      match_id   team  avg_team_kills  avg_team_deaths  avg_team_kd  \
0     34306992  Team0        9.048333         6.180000     1.495000   
1     34011901  Team1        7.958333         6.581667     1.243333   
2     34354163  Team1        8.770000         5.968333     1.535000   
3     35476864  Team0        6.731667         5.733333     1.191667   
4     35618237  Team0        8.015000         6.138333     1.456667   
...        ...    ...             ...              ...          ...   
1995  35376358  Team0        8.958000         6.486000     1.494000   
1996  35464066  Team1        8.326667         5.603333     1.673333   
1997  35003831  Team0        7.276667         6.251667     1.226667   
1998  34900932  Team1        8.038333         6.138333     1.321667   
1999  34548483  Team0        8.493333         5.961667     1.468333   

      avg_team_win_rate  avg_player_matches  avg_win_streak  avg_loss_streak  \
0             53.868333          864.166667        2.269058        

In [6]:

#-- Calculate hero trend statistics by team
hero_trend_by_team= con.execute(f"""
    WITH hero_trend_by_team AS (
        SELECT
            ta.match_id,
            ta.team,
            AVG(ht.win_rate) AS avg_hero_win_rate,
            AVG(ht.average_kd) AS avg_hero_kd,
            AVG(ht.pick_rate) AS avg_hero_pick_rate,
            SUM(ht.pick_rate) AS total_hero_popularity,
            MAX(ht.win_rate) AS max_hero_win_rate,
            MIN(ht.win_rate) AS min_hero_win_rate,
            MAX(ht.average_kd) AS max_hero_kd,
            MIN(ht.average_kd) AS min_hero_kd,
            STDDEV(ht.win_rate) AS hero_win_rate_variety,
            STDDEV(ht.average_kd) AS hero_kd_variety
        FROM 
            team_of_players ta
        JOIN 
            hero_trends ht ON ta.hero_id = ht.hero_id
        WHERE 
            ht.trend_window_days = 30  -- Using 30-day trends
        GROUP BY 
            ta.match_id, ta.team
    )
        SELECT * from hero_trend_by_team""").fetchdf()
print(hero_trend_by_team)

      match_id   team  avg_hero_win_rate  avg_hero_kd  avg_hero_pick_rate  \
0     34214833  Team0          48.426666     1.036667            3.653333   
1     34174977  Team1          49.271667     1.028333            3.655000   
2     34123189  Team0          49.111666     1.091667            4.260000   
3     35547102  Team1          49.600000     1.050000            4.516667   
4     35550830  Team1          48.000000     1.021667            3.860000   
...        ...    ...                ...          ...                 ...   
1995  34338969  Team0          48.978333     1.031667            4.225000   
1996  34300801  Team1          48.706666     1.095000            3.411667   
1997  34310865  Team1          48.690000     1.003333            4.120000   
1998  34258935  Team1          49.100000     1.013333            3.503333   
1999  34244973  Team1          50.568334     0.956667            3.463333   

      total_hero_popularity  max_hero_win_rate  min_hero_win_rate  \
0     

In [7]:
#-- Get player trends and statistics
recent_performance= con.execute(f"""
    WITH recent_performance AS (
    SELECT
        ta.match_id,
        ta.team,
        -- Existing rolling win/loss percentages
        AVG(prs.p_win_pct_2) AS avg_recent_win_pct_2,
        AVG(prs.p_win_pct_3) AS avg_recent_win_pct_3,
        AVG(prs.p_win_pct_4) AS avg_recent_win_pct_4,
        AVG(prs.p_win_pct_5) AS avg_recent_win_pct_5,
        AVG(prs.p_loss_pct_2) AS avg_recent_loss_pct_2,
        AVG(prs.p_loss_pct_3) AS avg_recent_loss_pct_3,
        AVG(prs.p_loss_pct_4) AS avg_recent_loss_pct_4,
        AVG(prs.p_loss_pct_5) AS avg_recent_loss_pct_5,
        MAX(prs.p_win_pct_5) AS max_recent_win_pct,
        MIN(prs.p_win_pct_5) AS min_recent_win_pct,
        
        -- Player-hero specific metrics
        AVG(prs.p_v_h_kd_pct) AS avg_hero_kd_percentage,
        AVG(prs.p_v_h_pick_rate) AS avg_hero_pick_percentage,
        AVG(prs.p_h_match_count) AS avg_hero_match_count,
        SUM(prs.p_h_match_count) AS total_hero_experience,
        MAX(prs.p_v_h_kd_pct) AS max_hero_kd_percentage,
        MIN(prs.p_v_h_kd_pct) AS min_hero_kd_percentage,
        STDDEV(prs.p_v_h_kd_pct) AS hero_kd_consistency
    FROM 
        team_of_players ta
    JOIN 
        player_rolling_stats prs ON ta.account_id = prs.account_id AND ta.match_id = prs.match_id
    GROUP BY 
        ta.match_id, ta.team
)
    Select * from recent_performance""").fetchdf()
print(recent_performance)

      match_id   team  avg_recent_win_pct_2  avg_recent_win_pct_3  \
0     34110066  Team0             33.333333             38.886668   
1     34627398  Team1             75.000000             66.668332   
2     34984449  Team1             50.000000             55.555000   
3     34611202  Team1             50.000000             44.443334   
4     34574209  Team1             50.000000             38.886668   
...        ...    ...                   ...                   ...   
1251  34687191  Team1             66.666667             61.113332   
1252  34075488  Team1             58.333333             55.556666   
1253  33855523  Team0             41.666667             38.888334   
1254  34422277  Team1             75.000000             55.556666   
1255  34905146  Team0             58.333333             61.113332   

      avg_recent_win_pct_4  avg_recent_win_pct_5  avg_recent_loss_pct_2  \
0                41.666667             50.000000              66.666667   
1                62.5

In [8]:
team_features = con.execute(f"""
    WITH team_features AS (
    SELECT
        ts.match_id,
        ts.team,
        -- Existing features from team stats
        ts.team_avg_kd,
        ts.team_max_kd,
        ts.team_min_kd,
        
        -- Existing features from player performance
        pp.avg_team_kills,
        pp.avg_team_deaths,
        pp.avg_team_kd,
        pp.avg_team_win_rate,
        pp.max_player_win_rate,
        pp.min_player_win_rate,
        pp.max_player_kd,
        pp.min_player_kd,
        pp.avg_player_matches,
        pp.total_team_experience,
        pp.most_experienced_player,
        pp.least_experienced_player,
        pp.win_rate_consistency,
        pp.kd_consistency,
        pp.avg_win_streak,
        pp.avg_loss_streak,
        pp.team_win_streaks_2plus,
        pp.team_win_streaks_3plus,
        pp.team_win_streaks_4plus,
        pp.team_win_streaks_5plus,
        pp.team_loss_streaks_2plus,
        pp.team_loss_streaks_3plus,
        pp.team_loss_streaks_4plus,
        pp.team_loss_streaks_5plus,
        
        -- Hero trend features
        ht.avg_hero_win_rate,
        ht.avg_hero_kd,
        ht.avg_hero_pick_rate,
        ht.total_hero_popularity,
        ht.max_hero_win_rate,
        ht.min_hero_win_rate,
        ht.max_hero_kd,
        ht.min_hero_kd,
        ht.hero_win_rate_variety,
        ht.hero_kd_variety,
        
        -- Recent performance metrics (traditional)
        COALESCE(rps.avg_recent_win_pct_2, 50) AS avg_recent_win_pct_2,
        COALESCE(rps.avg_recent_win_pct_3, 50) AS avg_recent_win_pct_3,
        COALESCE(rps.avg_recent_win_pct_4, 50) AS avg_recent_win_pct_4,
        COALESCE(rps.avg_recent_win_pct_5, 50) AS avg_recent_win_pct_5,
        COALESCE(rps.avg_recent_loss_pct_2, 50) AS avg_recent_loss_pct_2,
        COALESCE(rps.avg_recent_loss_pct_3, 50) AS avg_recent_loss_pct_3,
        COALESCE(rps.avg_recent_loss_pct_4, 50) AS avg_recent_loss_pct_4,
        COALESCE(rps.avg_recent_loss_pct_5, 50) AS avg_recent_loss_pct_5,
        COALESCE(rps.max_recent_win_pct, 50) AS max_recent_win_pct,
        COALESCE(rps.min_recent_win_pct, 50) AS min_recent_win_pct,
        
        -- NEW: Player-hero specific metrics from rolling stats
        COALESCE(rps.avg_hero_kd_percentage, 0) AS avg_hero_kd_percentage,
        COALESCE(rps.avg_hero_pick_percentage, 0) AS avg_hero_pick_percentage,
        COALESCE(rps.avg_hero_match_count, 0) AS avg_hero_match_count,
        COALESCE(rps.total_hero_experience, 0) AS total_hero_experience,
        COALESCE(rps.max_hero_kd_percentage, 0) AS max_hero_kd_percentage,
        COALESCE(rps.min_hero_kd_percentage, 0) AS min_hero_kd_percentage,
        COALESCE(rps.hero_kd_consistency, 0) AS hero_kd_consistency
    FROM 
        team_stats ts
    LEFT JOIN 
        player_performance pp ON ts.match_id = pp.match_id AND ts.team = pp.team
    LEFT JOIN 
        hero_trend_by_team ht ON ts.match_id = ht.match_id AND ts.team = ht.team
    LEFT JOIN 
        recent_performance rps ON ts.match_id = rps.match_id AND ts.team = rps.team
)SELECT * from team_features""").fetchdf()
print(team_features)


      match_id   team  team_avg_kd  team_max_kd  team_min_kd  avg_team_kills  \
0     34011901  Team1     1.970833     5.500000     0.375000        7.958333   
1     34354163  Team1     3.583333     8.000000     0.000000        8.770000   
2     34289712  Team0     0.837393     2.500000     0.250000        8.400000   
3     34079529  Team1     2.466667     7.000000     1.166667        8.458333   
4     34104396  Team0     3.616667     8.000000     1.000000        6.673333   
...        ...    ...          ...          ...          ...             ...   
1995  35199062  Team0     0.672222     1.750000     0.200000       10.756000   
1996  35450918  Team1     0.341270     0.666667     0.000000        7.988333   
1997  35248178  Team1     2.538889     7.000000     0.833333        9.311667   
1998  35560618  Team1     1.325321     3.200000     0.142857        7.483333   
1999  35239022  Team1     1.102393     1.909091     0.250000        8.413333   

      avg_team_deaths  avg_team_kd  avg

In [30]:
training_set = con.execute(f"""
WITH team_features_ordered AS (
    -- Ensure consistent ordering of teams
    SELECT 
        match_id, 
        team,
        team_avg_kd,
        team_max_kd,
        team_min_kd,
        avg_hero_kd_percentage,
        avg_hero_pick_percentage,
        avg_hero_match_count,
        total_hero_experience,
        max_hero_kd_percentage,
        min_hero_kd_percentage,
        ROW_NUMBER() OVER (PARTITION BY match_id ORDER BY team) AS team_order
    FROM 
        team_features
)

SELECT
    md.match_id,
    md.start_time,
    md.winning_team,
    
    -- Team 0 features (lower team value)
    t0.team_avg_kd AS t0_avg_kd,
    t0.team_max_kd AS t0_max_kd,
    t0.team_min_kd AS t0_min_kd,
    t0.avg_hero_kd_percentage AS t0_hero_kd_percentage,
    t0.avg_hero_pick_percentage AS t0_hero_pick_percentage,
    t0.avg_hero_match_count AS t0_hero_match_count,
    t0.total_hero_experience AS t0_hero_experience,
    t0.max_hero_kd_percentage AS t0_max_hero_kd_pct,
    t0.min_hero_kd_percentage AS t0_min_hero_kd_pct,
    
    -- Team 1 features (higher team value)
    t1.team_avg_kd AS t1_avg_kd,
    t1.team_max_kd AS t1_max_kd,
    t1.team_min_kd AS t1_min_kd,
    t1.avg_hero_kd_percentage AS t1_hero_kd_percentage,
    t1.avg_hero_pick_percentage AS t1_hero_pick_percentage,
    t1.avg_hero_match_count AS t1_hero_match_count,
    t1.total_hero_experience AS t1_hero_experience,
    t1.max_hero_kd_percentage AS t1_max_hero_kd_pct,
    t1.min_hero_kd_percentage AS t1_min_hero_kd_pct,
    
    -- Differential features
    (t0.team_avg_kd - t1.team_avg_kd) AS kd_diff,
    (t0.avg_hero_kd_percentage - t1.avg_hero_kd_percentage) AS hero_kd_pct_diff,
    
    -- Target variable: 1 if team with lower value won, 0 otherwise
    CASE WHEN md.winning_team = t0.team THEN 1 ELSE 0 END AS team0_won
FROM 
    match_data md
JOIN 
    team_features_ordered t0 ON md.match_id = t0.match_id AND t0.team_order = 1
JOIN 
    team_features_ordered t1 ON md.match_id = t1.match_id AND t1.team_order = 2
""").fetchdf()
print(f"Robust training set has {len(training_set)} rows")
print(training_set)

Robust training set has 1000 rows
     match_id          start_time winning_team  t0_avg_kd  t0_max_kd  \
0    34681243 2025-04-08 02:41:19        Team1   0.388889   0.800000   
1    35480510 2025-05-06 23:05:08        Team1   0.767226   2.000000   
2    33982008 2025-03-16 18:35:44        Team1   0.639983   1.333333   
3    34505619 2025-04-02 04:58:40        Team1   0.910317   2.800000   
4    34642415 2025-04-06 19:14:33        Team0   3.291667   7.000000   
..        ...                 ...          ...        ...        ...   
995  35200620 2025-04-26 20:32:51        Team0   2.222222   6.000000   
996  34071541 2025-03-19 16:27:32        Team1   1.555736   3.500000   
997  34662503 2025-04-07 13:22:53        Team1   1.480342   4.000000   
998  35320312 2025-05-01 05:47:08        Team0   1.199206   1.750000   
999  35448364 2025-05-05 18:24:57        Team1   0.631548   1.200000   

     t0_min_kd  t0_hero_kd_percentage  t0_hero_pick_percentage  \
0     0.000000              22.8333

In [31]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def train_match_prediction_model(data_path="data/final_training_data.csv", output_path="model_output/"):
    """
    Train a FastAI random forest model on the match prediction data
    
    Args:
        data_path: Path to the training data CSV
        output_path: Directory to save model outputs
        
    Returns:
        Trained FastAI model
    """
    # Create output directory
    os.makedirs(output_path, exist_ok=True)
    
    # Load the training data
    df = pd.read_csv(data_path)
    print(f"Loaded {len(df)} matches for training")
    
    # Check class balance
    class_balance = df['team0_won'].value_counts(normalize=True)
    print(f"Class balance: Team0 won {class_balance.get(1, 0):.2%}, Team1 won {class_balance.get(0, 0):.2%}")
    
    # Remove non-feature columns
    X = df.drop(['match_id', 'start_time', 'winning_team', 'team0_won'], axis=1)
    y = df['team0_won']

In [29]:
# Stage 1: Create a temporary view with just the match IDs and teams
stage1_query = """
CREATE OR REPLACE TEMPORARY VIEW stage1_matches AS
SELECT
    md.match_id,
    md.start_time,
    md.winning_team
FROM 
    match_data md
JOIN 
    team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
JOIN 
    team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
"""

con.execute(stage1_query)
stage1_count = con.execute("SELECT COUNT(*) FROM stage1_matches").fetchone()[0]
print(f"Stage 1: Created temporary view with {stage1_count} matches")

# Stage 2: Join with team0 features
stage2_query = """
CREATE OR REPLACE TEMPORARY VIEW stage2_matches AS
SELECT
    sm.*,
    t0.team_avg_kd AS t0_avg_kd,
    t0.team_max_kd AS t0_max_kd,
    t0.team_min_kd AS t0_min_kd,
    t0.avg_hero_kd_percentage AS t0_hero_kd_percentage,
    t0.avg_hero_pick_percentage AS t0_hero_pick_percentage,
    t0.avg_hero_match_count AS t0_hero_match_count,
    t0.total_hero_experience AS t0_hero_experience,
    t0.max_hero_kd_percentage AS t0_max_hero_kd_pct,
    t0.min_hero_kd_percentage AS t0_min_hero_kd_pct
FROM 
    stage1_matches sm
JOIN 
    team_features t0 ON sm.match_id = t0.match_id AND t0.team = 'Team0'
"""

con.execute(stage2_query)
stage2_count = con.execute("SELECT COUNT(*) FROM stage2_matches").fetchone()[0]
print(f"Stage 2: Added Team0 features, now have {stage2_count} matches")

# Stage 3: Join with team1 features
stage3_query = """
CREATE OR REPLACE TEMPORARY VIEW stage3_matches AS
SELECT
    sm.*,
    t1.team_avg_kd AS t1_avg_kd,
    t1.team_max_kd AS t1_max_kd,
    t1.team_min_kd AS t1_min_kd,
    t1.avg_hero_kd_percentage AS t1_hero_kd_percentage,
    t1.avg_hero_pick_percentage AS t1_hero_pick_percentage,
    t1.avg_hero_match_count AS t1_hero_match_count,
    t1.total_hero_experience AS t1_hero_experience,
    t1.max_hero_kd_percentage AS t1_max_hero_kd_pct,
    t1.min_hero_kd_percentage AS t1_min_hero_kd_pct
FROM 
    stage2_matches sm
JOIN 
    team_features t1 ON sm.match_id = t1.match_id AND t1.team = 'Team1'
"""

con.execute(stage3_query)
stage3_count = con.execute("SELECT COUNT(*) FROM stage3_matches").fetchone()[0]
print(f"Stage 3: Added Team1 features, now have {stage3_count} matches")

# Stage 4: Calculate differential features and target variable
final_dataset_query = """
SELECT
    match_id,
    start_time,
    winning_team,
    
    -- Team 0 features
    t0_avg_kd,
    t0_max_kd,
    t0_min_kd,
    t0_hero_kd_percentage,
    t0_hero_pick_percentage,
    t0_hero_match_count,
    t0_hero_experience,
    t0_max_hero_kd_pct,
    t0_min_hero_kd_pct,
    
    -- Team 1 features
    t1_avg_kd,
    t1_max_kd,
    t1_min_kd,
    t1_hero_kd_percentage,
    t1_hero_pick_percentage,
    t1_hero_match_count,
    t1_hero_experience,
    t1_max_hero_kd_pct,
    t1_min_hero_kd_pct,
    
    -- Differential features
    (t0_avg_kd - t1_avg_kd) AS kd_diff,
    (t0_hero_kd_percentage - t1_hero_kd_percentage) AS hero_kd_pct_diff,
    
    -- Target variable
    CASE WHEN winning_team = 'Team0' THEN 1 ELSE 0 END AS team0_won
FROM 
    stage3_matches
"""

final_dataset = con.execute(final_dataset_query).fetchdf()
print(f"Final dataset has {len(final_dataset)} rows")
print(final_dataset.head())

# Save the dataset if it has 1000 rows
if len(final_dataset) == 1000:
    final_dataset.to_csv("data/final_training_data.csv", index=False)
    print("Saved final training data to CSV")

Stage 1: Created temporary view with 1000 matches
Stage 2: Added Team0 features, now have 1000 matches
Stage 3: Added Team1 features, now have 1000 matches
Final dataset has 1000 rows
   match_id          start_time winning_team  t0_avg_kd  t0_max_kd  t0_min_kd  \
0  34011901 2025-03-17 17:19:24        Team1   0.769261   1.166667   0.333333   
1  34354163 2025-03-28 12:43:50        Team1   0.450000   2.000000   0.000000   
2  34079529 2025-03-19 20:49:48        Team1   0.571759   1.250000   0.000000   
3  34104752 2025-03-20 18:18:11        Team1   0.702617   1.250000   0.181818   
4  34574209 2025-04-04 18:11:35        Team0   4.798611  20.000000   0.500000   

   t0_hero_kd_percentage  t0_hero_pick_percentage  t0_hero_match_count  \
0              15.666667              1384.333333           149.666667   
1              37.166667               747.500000            79.833333   
2              12.500000              3118.833333           278.500000   
3             -10.333333         

In [20]:
successful_joins = """
SELECT 
    md.match_id,
    md.winning_team,
    t0.team AS team0,
    t1.team AS team1
FROM 
    match_data md
JOIN 
    team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
JOIN 
    team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
"""

successful_matches = con.execute(successful_joins).fetchdf()
print("\nSuccessful join matches:")
print(successful_matches)

# Now, let's check a few match_ids that don't join successfully
failed_joins = """
WITH successful_matches AS (
    SELECT md.match_id
    FROM match_data md
    JOIN team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
    JOIN team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
)

SELECT 
    md.match_id
FROM 
    match_data md
LEFT JOIN 
    successful_matches sm ON md.match_id = sm.match_id
WHERE 
    sm.match_id IS NULL
LIMIT 5
"""

failed_matches = con.execute(failed_joins).fetchdf()
print("\nFailed join matches:")
print(failed_matches)


Successful join matches:
     match_id winning_team  team0  team1
0    34681243        Team1  Team0  Team1
1    35480510        Team1  Team0  Team1
2    33982008        Team1  Team0  Team1
3    34505619        Team1  Team0  Team1
4    34642415        Team0  Team0  Team1
..        ...          ...    ...    ...
995  35200620        Team0  Team0  Team1
996  34071541        Team1  Team0  Team1
997  34662503        Team1  Team0  Team1
998  35320312        Team0  Team0  Team1
999  35448364        Team1  Team0  Team1

[1000 rows x 4 columns]

Failed join matches:
Empty DataFrame
Columns: [match_id]
Index: []


In [23]:
detailed_team_values = """
SELECT 
    team,
    LENGTH(team) AS len,
    HEX(team) AS hex_value,
    COUNT(*) AS count
FROM 
    team_features
GROUP BY 
    team, LENGTH(team), HEX(team)
"""

detailed_values = con.execute(detailed_team_values).fetchdf()
print("\nDetailed team values:")
print(detailed_values)


Detailed team values:
    team  len   hex_value  count
0  Team1    5  5465616D31   1000
1  Team0    5  5465616D30   1000


In [25]:
null_check_query = """
SELECT
    SUM(CASE WHEN t0.avg_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t0_null_hero_kd_pct,
    SUM(CASE WHEN t0.avg_hero_pick_percentage IS NULL THEN 1 ELSE 0 END) AS t0_null_hero_pick_pct,
    SUM(CASE WHEN t0.avg_hero_match_count IS NULL THEN 1 ELSE 0 END) AS t0_null_hero_match_count,
    SUM(CASE WHEN t0.total_hero_experience IS NULL THEN 1 ELSE 0 END) AS t0_null_hero_experience,
    SUM(CASE WHEN t0.max_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t0_null_max_hero_kd_pct,
    SUM(CASE WHEN t0.min_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t0_null_min_hero_kd_pct,
    
    SUM(CASE WHEN t1.avg_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t1_null_hero_kd_pct,
    SUM(CASE WHEN t1.avg_hero_pick_percentage IS NULL THEN 1 ELSE 0 END) AS t1_null_hero_pick_pct,
    SUM(CASE WHEN t1.avg_hero_match_count IS NULL THEN 1 ELSE 0 END) AS t1_null_hero_match_count,
    SUM(CASE WHEN t1.total_hero_experience IS NULL THEN 1 ELSE 0 END) AS t1_null_hero_experience,
    SUM(CASE WHEN t1.max_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t1_null_max_hero_kd_pct,
    SUM(CASE WHEN t1.min_hero_kd_percentage IS NULL THEN 1 ELSE 0 END) AS t1_null_min_hero_kd_pct
FROM 
    match_data md
JOIN 
    team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
JOIN 
    team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
"""

null_counts = con.execute(null_check_query).fetchdf()
print("\nNULL counts in key columns:")
print(null_counts)


NULL counts in key columns:
   t0_null_hero_kd_pct  t0_null_hero_pick_pct  t0_null_hero_match_count  \
0                  0.0                    0.0                       0.0   

   t0_null_hero_experience  t0_null_max_hero_kd_pct  t0_null_min_hero_kd_pct  \
0                      0.0                      0.0                      0.0   

   t1_null_hero_kd_pct  t1_null_hero_pick_pct  t1_null_hero_match_count  \
0                  0.0                    0.0                       0.0   

   t1_null_hero_experience  t1_null_max_hero_kd_pct  t1_null_min_hero_kd_pct  
0                      0.0                      0.0                      0.0  


In [27]:
minimalist_query = """
SELECT
    md.match_id,
    md.start_time,
    md.winning_team,
    
    -- Only essential Team 0 features
    t0.team_avg_kd AS t0_avg_kd,
    t0.team_max_kd AS t0_max_kd,
    t0.team_min_kd AS t0_min_kd,
    
    -- Only essential Team 1 features
    t1.team_avg_kd AS t1_avg_kd,
    t1.team_max_kd AS t1_max_kd,
    t1.team_min_kd AS t1_min_kd,
    
    -- Differential features
    (t0.team_avg_kd - t1.team_avg_kd) AS kd_diff,
    
    -- Target variable
    CASE WHEN md.winning_team = 'Team0' THEN 1 ELSE 0 END AS team0_won
FROM 
    match_data md
JOIN 
    team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
JOIN 
    team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
"""

minimalist_result = con.execute(minimalist_query).fetchdf()
print(f"\nMinimalist query returned {len(minimalist_result)} rows")

# If this works, save it to a CSV file
if len(minimalist_result) > 5:
    minimalist_result.to_csv("data/minimalist_training_data.csv", index=False)
    print("Saved minimalist training data to CSV")


Minimalist query returned 1000 rows
Saved minimalist training data to CSV


In [28]:
if len(minimalist_result) > 5:
    # Start adding one field at a time
    for field in [
        "t0.avg_hero_kd_percentage AS t0_hero_kd_percentage",
        "t0.avg_hero_pick_percentage AS t0_hero_pick_percentage",
        "t0.avg_hero_match_count AS t0_hero_match_count",
        "t0.total_hero_experience AS t0_hero_experience",
        "t0.max_hero_kd_percentage AS t0_max_hero_kd_pct",
        "t0.min_hero_kd_percentage AS t0_min_hero_kd_pct",
        "t1.avg_hero_kd_percentage AS t1_hero_kd_percentage",
        "t1.avg_hero_pick_percentage AS t1_hero_pick_percentage",
        "t1.avg_hero_match_count AS t1_hero_match_count",
        "t1.total_hero_experience AS t1_hero_experience",
        "t1.max_hero_kd_percentage AS t1_max_hero_kd_pct",
        "t1.min_hero_kd_percentage AS t1_min_hero_kd_pct"
    ]:
        # Create a new query with this field added
        current_fields = minimalist_query.split("FROM")[0] + f", {field}\nFROM" + minimalist_query.split("FROM")[1]
        
        # Test it
        try:
            field_count = con.execute(f"SELECT COUNT(*) FROM ({current_fields})").fetchone()[0]
            print(f"Adding field {field}: {field_count} rows")
            
            if field_count == 5:
                print(f"Found the problem field: {field}")
                break
        except Exception as e:
            print(f"Error with field {field}: {str(e)}")
            break

Adding field t0.avg_hero_kd_percentage AS t0_hero_kd_percentage: 1000 rows
Adding field t0.avg_hero_pick_percentage AS t0_hero_pick_percentage: 1000 rows
Adding field t0.avg_hero_match_count AS t0_hero_match_count: 1000 rows
Adding field t0.total_hero_experience AS t0_hero_experience: 1000 rows
Adding field t0.max_hero_kd_percentage AS t0_max_hero_kd_pct: 1000 rows
Adding field t0.min_hero_kd_percentage AS t0_min_hero_kd_pct: 1000 rows
Adding field t1.avg_hero_kd_percentage AS t1_hero_kd_percentage: 1000 rows
Adding field t1.avg_hero_pick_percentage AS t1_hero_pick_percentage: 1000 rows
Adding field t1.avg_hero_match_count AS t1_hero_match_count: 1000 rows
Adding field t1.total_hero_experience AS t1_hero_experience: 1000 rows
Adding field t1.max_hero_kd_percentage AS t1_max_hero_kd_pct: 1000 rows
Adding field t1.min_hero_kd_percentage AS t1_min_hero_kd_pct: 1000 rows


In [11]:
# Check for some sample match IDs that are in match_data but don't successfully join
sample_missing_matches_query = """
WITH successful_matches AS (
    SELECT md.match_id
    FROM match_data md
    JOIN team_features t0 ON md.match_id = t0.match_id AND t0.team = 'Team0'
    JOIN team_features t1 ON md.match_id = t1.match_id AND t1.team = 'Team1'
)
SELECT md.match_id
FROM match_data md
LEFT JOIN successful_matches sm ON md.match_id = sm.match_id
WHERE sm.match_id IS NULL
LIMIT 5
"""

missing_matches = con.execute(sample_missing_matches_query).fetchdf()
print("Sample match IDs that don't join successfully:")
print(missing_matches)

# For a sample missing match, check if it exists in team_features
if len(missing_matches) > 0:
    sample_match_id = missing_matches['match_id'].iloc[0]
    team_features_check_query = f"""
    SELECT match_id, team
    FROM team_features
    WHERE match_id = '{sample_match_id}'
    """
    team_features_check = con.execute(team_features_check_query).fetchdf()
    print(f"\nTeam features for match {sample_match_id}:")
    print(team_features_check)

Sample match IDs that don't join successfully:
Empty DataFrame
Columns: [match_id]
Index: []


In [ ]:
coverage = pd.DataFrame({
    'Total': [total_counts['total_player_matches'][0], 
             total_counts['total_matches'][0], 
             total_counts['total_players'][0]],
    'Covered': [covered_counts['covered_player_matches'][0], 
               covered_counts['covered_matches'][0], 
               covered_counts['covered_players'][0]],
})
coverage['Coverage %'] = (coverage['Covered'] / coverage['Total'] * 100).round(2)
coverage.index = ['Player-Match Combinations', 'Unique Matches', 'Unique Players']

print("Rolling Stats Coverage:")
print(coverage)

In [ ]:
missing_players = con.execute(f"""
    SELECT DISTINCT ta.account_id from team_of_players ta""").fetchdf()
print(missing_players)

In [ ]:
import services.orchestrators as o
ids = missing_players['account_id'].unique().tolist()
small_df = o.run_etl_player_hero_match_trends_from_db(account_ids=ids)

In [ ]:

        # Execute the complete query
try:
    result_df = pd.read_sql_query(complete_query, con)
    print(f"Successfully fetched {len(result_df)} matches for training")
    print(result_df.head())
except Exception as e:
    print(f"Error executing query: {e}")




In [ ]:
team_values = con.execute("SELECT DISTINCT team FROM team_features").fetchdf()
print("Team values in team_features:")
print(team_values)

# Check winning team values in match_data
winning_values = con.execute("SELECT DISTINCT winning_team FROM match_data").fetchdf()
print("Winning team values in match_data:")
print(winning_values)

# Check a specific match as an example
sample_match = con.execute("SELECT match_id FROM match_data LIMIT 1").fetchone()[0]
match_teams = con.execute(f"""
SELECT match_id, team FROM team_features WHERE match_id = '{sample_match}'
""").fetchdf()
print(f"\nTeams for match {sample_match}:")
print(match_teams)

match_winner = con.execute(f"""
SELECT match_id, winning_team FROM match_data WHERE match_id = '{sample_match}'
""").fetchdf()
print(f"\nWinner for match {sample_match}:")
print(match_winner)

In [ ]:
simplified_query = """
SELECT 
    md.match_id, 
    md.winning_team,
    t0.team AS team0,
    t1.team AS team1
FROM 
    match_data md
JOIN 
    team_features t0 ON md.match_id = t0.match_id AND t0.team = '0'
JOIN 
    team_features t1 ON md.match_id = t1.match_id AND t1.team = '1'
"""
simplified_result = con.execute(simplified_query).fetchdf()
print(f"Simplified query returned {len(simplified_result)} rows")

In [ ]:
# Check if all matches have both teams '0' and '1'
team_coverage_query = """
WITH team_counts AS (
    SELECT
        match_id,
        COUNT(DISTINCT CASE WHEN team = '0' THEN 1 END) AS has_team0,
        COUNT(DISTINCT CASE WHEN team = '1' THEN 1 END) AS has_team1
    FROM
        team_features
    GROUP BY
        match_id
)
SELECT
    SUM(CASE WHEN has_team0 > 0 AND has_team1 > 0 THEN 1 ELSE 0 END) AS complete_matches,
    SUM(CASE WHEN has_team0 > 0 AND has_team1 = 0 THEN 1 ELSE 0 END) AS team0_only,
    SUM(CASE WHEN has_team0 = 0 AND has_team1 > 0 THEN 1 ELSE 0 END) AS team1_only,
    COUNT(*) AS total_matches
FROM
    team_counts
"""
team_coverage = con.execute(team_coverage_query).fetchdf()
print("Match team coverage:")
print(team_coverage)

In [ ]:
# Check if team values are strings or numbers
team_types_query = """
SELECT 
    team, 
    TYPEOF(team) AS team_type,
    COUNT(*) AS count
FROM 
    team_features
GROUP BY 
    team, TYPEOF(team)
"""
team_types = con.execute(team_types_query).fetchdf()
print("Team value types:")
print(team_types)

winning_types_query = """
SELECT 
    winning_team, 
    TYPEOF(winning_team) AS winning_type,
    COUNT(*) AS count
FROM 
    match_data
GROUP BY 
    winning_team, TYPEOF(winning_team)
"""
winning_types = con.execute(winning_types_query).fetchdf()
print("Winning team value types:")
print(winning_types)

In [ ]:
import pandas as pd
import duckdb
from sklearn.model_selection import train_test_split
from fastai.tabular.all import *
import matplotlib.pyplot as plt
import numpy as np
from services import model_queries as mq

# Connect to database
con = duckdb.connect("data/deadlock.db")

# Execute your SQL query to get the structured data
df = mq.fetch_training_data(con)

# Check the data
print(f"Dataset shape: {df.shape}")
print(f"Sample of data:\n{df.head()}")

#check balance of the target variable
print(f"Target distribution:\n{df['team0_won'].value_counts(normalize=True)}")

In [ ]:
# Remove non-feature columns
feature_df = training_data.drop(['match_id', 'start_time', 'winning_team'], axis=1)

# Define target variable
target = 'team0_won'

# Split into features and target
X = feature_df.drop(target, axis=1)
y = feature_df[target]

# Split into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Create FastAI TabularPandas object
procs = [Categorify, FillMissing, Normalize]
splits = (list(range(len(X_train))), list(range(len(X_train), len(X_train) + len(X_valid))))
to = TabularPandas(pd.concat([X_train, X_valid]), procs, 
                   cat_names=[], 
                   cont_names=list(X_train.columns),
                   y_names=target, 
                   splits=splits)

# Create DataLoaders
dls = to.dataloaders(bs=64)

In [ ]:
# Create and train the random forest model
learn = tabular_learner(dls, metrics=accuracy)

# Find optimal learning rate
learn.lr_find()

# Train the model
learn.fit_one_cycle(5, 1e-2)

# Save the model
learn.export('deadlock_match_prediction_model.pkl')

In [ ]:
# Evaluate model performance
interp = ClassificationInterpretation.from_learner(learn)

# Confusion matrix
interp.plot_confusion_matrix()

# Feature importance
importance = interp.feature_importance()
plt.figure(figsize=(12, 8))
plt.barh(X.columns[importance[1]], importance[0])
plt.xlabel('Importance')
plt.title('Feature Importance')
plt.tight_layout()
plt.show()

# Top losses (most incorrect predictions)
interp.plot_top_losses(10)